# Obsprocessors

> Processors for observations can be used to process the input for an agent before it is being passed to the agent. 

In [ ]:
#| default_exp obsprocessors

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from typing import Union, Optional

import numpy as np
from ddopnew.utils import Parameter, check_parameter_types

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#| export

class FlattenTimeDimNumpy():

    """
    Preprocessor to flatten the time and feature dimension of the input.
    Used, e.g., to convert time-series data for models that cannot process
    a time dimension such as MLPs or Regression models.
    """

    def __init__(self,
        allow_2d: Optional[bool] = False, #
        batch_dim_included: Optional[bool] = True
        ):
        self.allow_2d = allow_2d
        self.batch_dim_included = batch_dim_included

    def check_input(self,
            input: np.ndarray #
            ):
        """
        Check that the input is a PyTorch tensor with the correct shape.
        """
        # Check if the input is a PyTorch tensor
        if not isinstance(input, np.ndarray):
            raise TypeError(f"Expected input to be a numpy array, but got {type(input)} instead.")

        # Determine expected dimensions based on batch_dim_included
        expected_ndim = 3 if self.batch_dim_included else 2
        allow_ndim = 2 if self.batch_dim_included else 1

        # Check if the input array has the correct dimensions
        if input.ndim == expected_ndim:
            # If the input is 3D, it is valid regardless of allow_2d
            return
        elif input.ndim == allow_ndim:
            # If the input has fewer dimensions, check if the reduced dimension is allowed
            if not self.allow_2d:
                raise ValueError(
                    f"Expected input to have {expected_ndim} dimensions with shape "
                    f"{'(batch_size, timesteps, features)' if self.batch_dim_included else '(timesteps, features)'}, "
                    f"but got shape {input.shape} instead. "
                    f"{allow_ndim}D inputs are not allowed when allow_2d is False."
                )
        else:
            # If the input has an unexpected number of dimensions
            expected_shape_msg = (
                f"Expected input to have {expected_ndim} dimensions with shape "
                f"{'(batch_size, timesteps, features)' if self.batch_dim_included else '(timesteps, features)'}"
                if not self.allow_2d
                else f"Expected input to have {expected_ndim} dimensions with shape "
                     f"{'(batch_size, timesteps, features)' if self.batch_dim_included else '(timesteps, features)'} "
                     f"or {allow_ndim} dimensions with shape "
                     f"{'(batch_size, features)' if self.batch_dim_included else '(features)'}"
            )
            raise ValueError(f"{expected_shape_msg}, but got shape {input.shape} instead.")
            
    def __call__(self,
                input: np.ndarray
                ) -> np.ndarray:
                
        """
        Process the input array by keeping the batch dimension and flattening
        the time and feature dimensions.
        """

        # Validate the input tensor
        self.check_input(input)

        if self.batch_dim_included:
            # If batch dimension is included
            if input.ndim == 2:
                output = input
            else:
                # Keep the batch dimension, flatten time and feature dimensions
                batch_size, timesteps, features = input.shape
                output = input.reshape(batch_size, -1)
        else:
            # If batch dimension is not included
            if input.ndim == 1:
                output = input
            else:
                # Flatten time and feature dimensions
                timesteps, features = input.shape
                output = input.reshape(-1)

        return output

In [ ]:
show_doc(FlattenTimeDimNumpy, title_level=2)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/obsprocessors.py#L17){target="_blank" style="float:right; font-size:smaller"}

## FlattenTimeDimNumpy

>      FlattenTimeDimNumpy (allow_2d:Optional[bool]=False,
>                           batch_dim_included:Optional[bool]=True)

*Preprocessor to flatten the time and feature dimension of the input.
Used, e.g., to convert time-series data for models that cannot process
a time dimension such as MLPs or Regression models.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| allow_2d | Optional | False |  |
| batch_dim_included | Optional | True |  |

In [ ]:
show_doc(FlattenTimeDimNumpy.check_input)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/obsprocessors.py#L32){target="_blank" style="float:right; font-size:smaller"}

### FlattenTimeDimNumpy.check_input

>      FlattenTimeDimNumpy.check_input (input:numpy.ndarray)

*Check that the input is a PyTorch tensor with the correct shape.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| input | ndarray |  |

In [ ]:
show_doc(FlattenTimeDimNumpy.__call__)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/obsprocessors.py#L72){target="_blank" style="float:right; font-size:smaller"}

### FlattenTimeDimNumpy.__call__

>      FlattenTimeDimNumpy.__call__ (input:numpy.ndarray)

*Process the input array by keeping the batch dimension and flattening
the time and feature dimensions.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()